In [1]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [6]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import os
import threading
from pathlib import Path

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Bulk Image Cropper")
        self.root.geometry("1200x800")
        
        # Variables
        self.selected_images = []
        self.current_image_index = 0
        self.current_image = None
        self.display_image = None
        self.crop_coords = None
        self.is_cropping = False
        self.start_x = self.start_y = 0
        self.rect_id = None
        self.scale_factor = 1.0
        
        # Load face cascade
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
        
        self.setup_ui()
        
    def setup_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left panel - Controls
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # File selection
        file_frame = ttk.LabelFrame(left_frame, text="Image Selection", padding=10)
        file_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(file_frame, text="Select Images", command=self.select_images).pack(fill=tk.X, pady=2)
        ttk.Button(file_frame, text="Select Folder", command=self.select_folder).pack(fill=tk.X, pady=2)
        
        self.image_count_label = ttk.Label(file_frame, text="No images selected")
        self.image_count_label.pack(pady=5)
        
        # Manual cropping controls
        manual_frame = ttk.LabelFrame(left_frame, text="Manual Cropping", padding=10)
        manual_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(manual_frame, text="Previous Image", command=self.prev_image).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Next Image", command=self.next_image).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Crop Current", command=self.crop_current_manual).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Reset Crop", command=self.reset_crop).pack(fill=tk.X, pady=2)
        
        # Manual crop info
        ttk.Label(manual_frame, text="Manual crops preserve exact", font=('TkDefaultFont', 8)).pack(pady=(5,0))
        ttk.Label(manual_frame, text="bounding box dimensions", font=('TkDefaultFont', 8)).pack()
        
        self.current_image_label = ttk.Label(manual_frame, text="No image loaded")
        self.current_image_label.pack(pady=5)
        
        # Automatic cropping controls
        auto_frame = ttk.LabelFrame(left_frame, text="Automatic Cropping", padding=10)
        auto_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.crop_mode = tk.StringVar(value="face")
        ttk.Radiobutton(auto_frame, text="Face Detection", variable=self.crop_mode, value="face").pack(anchor=tk.W)
        ttk.Radiobutton(auto_frame, text="Body Detection", variable=self.crop_mode, value="body").pack(anchor=tk.W)
        ttk.Radiobutton(auto_frame, text="Center Crop", variable=self.crop_mode, value="center").pack(anchor=tk.W)
        
        # Crop size settings
        size_frame = ttk.Frame(auto_frame)
        size_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(size_frame, text="Width:").grid(row=0, column=0, sticky=tk.W)
        self.width_var = tk.StringVar(value="700")
        ttk.Entry(size_frame, textvariable=self.width_var, width=8).grid(row=0, column=1, padx=5)
        
        ttk.Label(size_frame, text="Height:").grid(row=1, column=0, sticky=tk.W)
        self.height_var = tk.StringVar(value="700")
        ttk.Entry(size_frame, textvariable=self.height_var, width=8).grid(row=1, column=1, padx=5)
        
        ttk.Button(auto_frame, text="Auto Crop All", command=self.auto_crop_all).pack(fill=tk.X, pady=5)
        
        # Auto crop info
        ttk.Label(auto_frame, text="Auto crops resize to specified", font=('TkDefaultFont', 8)).pack(pady=(5,0))
        ttk.Label(auto_frame, text="width/height dimensions", font=('TkDefaultFont', 8)).pack()
        
        # Output settings
        output_frame = ttk.LabelFrame(left_frame, text="Output Settings", padding=10)
        output_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(output_frame, text="Select Output Folder", command=self.select_output_folder).pack(fill=tk.X, pady=2)
        
        self.output_folder = tk.StringVar(value="./cropped_images")
        ttk.Label(output_frame, text="Output:").pack(anchor=tk.W)
        ttk.Label(output_frame, textvariable=self.output_folder, wraplength=200).pack(anchor=tk.W)
        
        # Progress bar
        self.progress = ttk.Progressbar(left_frame, mode='determinate')
        self.progress.pack(fill=tk.X, pady=10)
        
        # Right panel - Image display
        right_frame = ttk.Frame(main_frame)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Canvas for image display
        self.canvas = tk.Canvas(right_frame, bg='white', cursor='cross')
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Bind mouse events for manual cropping
        self.canvas.bind("<Button-1>", self.start_crop)
        self.canvas.bind("<B1-Motion>", self.draw_crop)
        self.canvas.bind("<ButtonRelease-1>", self.end_crop)
        
        # Status bar
        self.status_label = ttk.Label(self.root, text="Ready", relief=tk.SUNKEN)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X)
        
    def select_images(self):
        try:
            files = filedialog.askopenfilenames(
                title="Select Images",
                filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif")]
            )
            if files:  # Only proceed if files were actually selected
                self.selected_images = list(files)
                self.update_image_count()
                self.current_image_index = 0
                self.load_current_image()
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting images: {str(e)}")
            
    def select_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select Image Folder")
            if folder:
                image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
                self.selected_images = []
                for file_path in Path(folder).rglob('*'):
                    if file_path.suffix.lower() in image_extensions:
                        self.selected_images.append(str(file_path))
                self.update_image_count()
                if self.selected_images:
                    self.current_image_index = 0
                    self.load_current_image()
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting folder: {str(e)}")
                
    def select_output_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select Output Folder")
            if folder:
                self.output_folder.set(folder)
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting output folder: {str(e)}")
            
    def update_image_count(self):
        count = len(self.selected_images)
        self.image_count_label.config(text=f"{count} images selected")
        
    def load_current_image(self):
        if not self.selected_images:
            return
            
        try:
            image_path = self.selected_images[self.current_image_index]




            #load with pil first to get DPI
            pil_img = Image.open(image_path)
            self.current_image = cv2.imread(image_path)
            
            if self.current_image is None:
                messagebox.showerror("Error", f"Could not load image: {image_path}")
                return
                
            # Update current image label
            filename = os.path.basename(image_path)
            self.current_image_label.config(text=f"{self.current_image_index + 1}/{len(self.selected_images)}: {filename}")
            
            # Display image
            self.display_image_on_canvas()
        except Exception as e:
            messagebox.showerror("Error", f"Error loading image: {str(e)}")
        
    def display_image_on_canvas(self):
        if self.current_image is None:
            return
            
        try:
            # Check if root window still exists
            if not self.root.winfo_exists():
                return
                
            # Convert BGR to RGB
            rgb_image = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
            
            # Calculate scale factor to fit canvas
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            if canvas_width <= 1 or canvas_height <= 1:
                self.root.after(100, self.display_image_on_canvas)
                return
                
            img_height, img_width = rgb_image.shape[:2]
            
            scale_x = canvas_width / img_width
            scale_y = canvas_height / img_height
            self.scale_factor = min(scale_x, scale_y, 1.0)
            
            new_width = int(img_width * self.scale_factor)
            new_height = int(img_height * self.scale_factor)
            
            # Resize image
            resized_image = cv2.resize(rgb_image, (new_width, new_height))
            
            # Convert to PIL Image
            pil_image = Image.fromarray(resized_image)
            
            # Create PhotoImage with explicit master reference
            self.display_image = ImageTk.PhotoImage(pil_image, master=self.root)
            
            # Clear canvas and display image
            self.canvas.delete("all")
            self.canvas.create_image(canvas_width//2, canvas_height//2, image=self.display_image)
            
        except Exception as e:
            print(f"Error displaying image: {str(e)}")
            # Try to show error message if root still exists
            try:
                if self.root.winfo_exists():
                    messagebox.showerror("Error", f"Error displaying image: {str(e)}")
            except:
                pass
        
    def start_crop(self, event):
        self.is_cropping = True
        self.start_x = event.x
        self.start_y = event.y
        if self.rect_id:
            self.canvas.delete(self.rect_id)
            
    def draw_crop(self, event):
        if self.is_cropping:
            if self.rect_id:
                self.canvas.delete(self.rect_id)
            self.rect_id = self.canvas.create_rectangle(
                self.start_x, self.start_y, event.x, event.y,
                outline='red', width=2
            )
            
    def end_crop(self, event):
        if self.is_cropping:
            self.is_cropping = False
            
            # Calculate crop coordinates in original image
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            # Get image position on canvas
            img_height, img_width = self.current_image.shape[:2]
            scaled_width = int(img_width * self.scale_factor)
            scaled_height = int(img_height * self.scale_factor)
            
            img_x = (canvas_width - scaled_width) // 2
            img_y = (canvas_height - scaled_height) // 2
            
            # Convert canvas coordinates to image coordinates
            x1 = max(0, int((self.start_x - img_x) / self.scale_factor))
            y1 = max(0, int((self.start_y - img_y) / self.scale_factor))
            x2 = min(img_width, int((event.x - img_x) / self.scale_factor))
            y2 = min(img_height, int((event.y - img_y) / self.scale_factor))
            
            if x2 > x1 and y2 > y1:
                self.crop_coords = (x1, y1, x2, y2)
            else:
                self.crop_coords = None
                if self.rect_id:
                    self.canvas.delete(self.rect_id)
                    self.rect_id = None
                

                
    def reset_crop(self):
        if self.rect_id:
            self.canvas.delete(self.rect_id)
            self.rect_id = None
        self.crop_coords = None



        
    def crop_current_manual(self):
        if self.current_image is not None and self.crop_coords:
            try:
                # Get the crop coordinates in original image dimensions
                x1, y1, x2, y2 = self.crop_coords
                
                # Ensure coordinates are within image bounds
                img_height, img_width = self.current_image.shape[:2]
                x1 = max(0, min(x1, img_width - 1))
                y1 = max(0, min(y1, img_height - 1))
                x2 = max(0, min(x2, img_width))
                y2 = max(0, min(y2, img_height))
                
                # Ensure valid dimensions
                if x2 > x1 and y2 > y1:
                    cropped = self.current_image[y1:y2, x1:x2]
                    self.save_cropped_image_manual(cropped, self.current_image_index)
                    messagebox.showinfo("Success", "Image cropped and saved!")
                else:
                    messagebox.showwarning("Warning", "Invalid crop area selected!")
            except Exception as e:
                messagebox.showerror("Error", f"Error cropping image: {str(e)}")
        else:
            messagebox.showwarning("Warning", "Please select a crop area first!")


            
    def prev_image(self):
        if self.selected_images and self.current_image_index > 0:
            try:
                self.current_image_index -= 1
                self.load_current_image()
                self.reset_crop()
            except Exception as e:
                messagebox.showerror("Error", f"Error loading previous image: {str(e)}")
            
    def next_image(self):
        if self.selected_images and self.current_image_index < len(self.selected_images) - 1:
            try:
                self.current_image_index += 1
                self.load_current_image()
                self.reset_crop()
            except Exception as e:
                messagebox.showerror("Error", f"Error loading next image: {str(e)}")
            
    def auto_crop_all(self):
        if not self.selected_images:
            messagebox.showwarning("Warning", "Please select images first!")
            return
            
        # Create output directory
        output_dir = self.output_folder.get()
        os.makedirs(output_dir, exist_ok=True)
        
        # Start processing in a separate thread
        thread = threading.Thread(target=self.process_images_thread)
        thread.daemon = True
        thread.start()
        
    def process_images_thread(self):
        total_images = len(self.selected_images)
        self.progress['maximum'] = total_images
        
        try:
            target_width = int(self.width_var.get())
            target_height = int(self.height_var.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid width and height values!")
            return
            
        success_count = 0
        
        for i, image_path in enumerate(self.selected_images):
            try:
                # Update status
                self.root.after(0, lambda: self.status_label.config(text=f"Processing {i+1}/{total_images}"))
                
                # Load image
                image = cv2.imread(image_path)
                if image is None:
                    continue
                    
                # Apply auto cropping based on mode
                mode = self.crop_mode.get()
                cropped_image = self.auto_crop_image(image, mode, target_width, target_height)
                
                if cropped_image is not None:
                    self.save_cropped_image(cropped_image, i)
                    success_count += 1
                    
                # Update progress
                self.root.after(0, lambda: self.progress.config(value=i+1))
                
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
                continue
                
        # Show completion message
        self.root.after(0, lambda: messagebox.showinfo("Complete", f"Processed {success_count}/{total_images} images successfully!"))
        self.root.after(0, lambda: self.status_label.config(text="Ready"))
        self.root.after(0, lambda: self.progress.config(value=0))
        
    def auto_crop_image(self, image, mode, target_width, target_height):
        height, width = image.shape[:2]
        
        if mode == "face":
            # Face detection
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
            
            if len(faces) > 0:
                # Get the largest face
                largest_face = max(faces, key=lambda x: x[2] * x[3])
                x, y, w, h = largest_face
                
                # Expand crop area around face
                padding = max(w, h) // 2
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(width, x + w + padding)
                y2 = min(height, y + h + padding)
                
                cropped = image[y1:y2, x1:x2]
                return cv2.resize(cropped, (target_width, target_height))
                
        elif mode == "body":
            # Body detection
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            bodies = self.body_cascade.detectMultiScale(gray, 1.1, 4)
            
            if len(bodies) > 0:
                # Get the largest body
                largest_body = max(bodies, key=lambda x: x[2] * x[3])
                x, y, w, h = largest_body
                
                cropped = image[y:y+h, x:x+w]
                return cv2.resize(cropped, (target_width, target_height))
                
        # Default to center crop
        aspect_ratio = target_width / target_height
        
        if width / height > aspect_ratio:
            # Image is wider - crop width
            new_width = int(height * aspect_ratio)
            x_offset = (width - new_width) // 2
            cropped = image[:, x_offset:x_offset + new_width]
        else:
            # Image is taller - crop height
            new_height = int(width / aspect_ratio)
            y_offset = (height - new_height) // 2
            cropped = image[y_offset:y_offset + new_height, :]
            
        return cv2.resize(cropped, (target_width, target_height))
        
    def save_cropped_image_manual(self, cropped_image, index):
        """Save manually cropped image without resizing - preserves exact crop dimensions"""
        try:
            output_dir = self.output_folder.get()
            os.makedirs(output_dir, exist_ok=True)
            
            original_path = self.selected_images[index]
            filename = os.path.basename(original_path)
            name, ext = os.path.splitext(filename)
            
            # Add manual crop suffix to distinguish from auto-cropped images
            output_path = os.path.join(output_dir, f"{name}_manual_crop{ext}")
            success = cv2.imwrite(output_path, cropped_image)
            
            if not success:
                raise Exception(f"Failed to save image to {output_path}")
                
        except Exception as e:
            print(f"Error saving manually cropped image: {str(e)}")
            raise e
    
    def save_cropped_image(self, cropped_image, index):
        """Save automatically cropped image with resizing applied"""
        try:
            output_dir = self.output_folder.get()
            os.makedirs(output_dir, exist_ok=True)
            
            original_path = self.selected_images[index]
            filename = os.path.basename(original_path)
            name, ext = os.path.splitext(filename)
            
            # Add auto crop suffix to distinguish from manually cropped images
            output_path = os.path.join(output_dir, f"{name}_auto_crop{ext}")
            success = cv2.imwrite(output_path, cropped_image)
            
            if not success:
                raise Exception(f"Failed to save image to {output_path}")
                
        except Exception as e:
            print(f"Error saving cropped image: {str(e)}")
            raise e

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = ImageCropper(root)
        root.mainloop()
    except Exception as e:
        print(f"Error starting application: {str(e)}")
        import traceback
        traceback.print_exc()

In [ ]:
#try

In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import os
import threading
from pathlib import Path
from tkinter import colorchooser



class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Bulk Image Cropper")
        self.root.geometry("1200x800")
        
        # Variables
        self.selected_images = []
        self.current_image_index = 0
        self.current_image = None
        self.display_image = None
        self.crop_coords = None
        self.is_cropping = False
        self.start_x = self.start_y = 0
        self.rect_id = None
        self.scale_factor = 1.0
        
        # Load face cascade
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
        
        self.setup_ui()
        
    def setup_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left panel - Controls
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # File selection
        file_frame = ttk.LabelFrame(left_frame, text="Image Selection", padding=10)
        file_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(file_frame, text="Select Images", command=self.select_images).pack(fill=tk.X, pady=2)
        ttk.Button(file_frame, text="Select Folder", command=self.select_folder).pack(fill=tk.X, pady=2)
        
        self.image_count_label = ttk.Label(file_frame, text="No images selected")
        self.image_count_label.pack(pady=5)
        
        # Manual cropping controls
        manual_frame = ttk.LabelFrame(left_frame, text="Manual Cropping", padding=10)
        manual_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(manual_frame, text="Previous Image", command=self.prev_image).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Next Image", command=self.next_image).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Crop Current", command=self.crop_current_manual).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Reset Crop", command=self.reset_crop).pack(fill=tk.X, pady=2)
        
        # Manual crop info
        ttk.Label(manual_frame, text="Manual crops preserve exact", font=('TkDefaultFont', 8)).pack(pady=(5,0))
        ttk.Label(manual_frame, text="bounding box dimensions", font=('TkDefaultFont', 8)).pack()
        
        self.current_image_label = ttk.Label(manual_frame, text="No image loaded")
        self.current_image_label.pack(pady=5)
        
        # Automatic cropping controls
        auto_frame = ttk.LabelFrame(left_frame, text="Automatic Cropping", padding=10)
        auto_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.crop_mode = tk.StringVar(value="face")
        ttk.Radiobutton(auto_frame, text="Face Detection", variable=self.crop_mode, value="face").pack(anchor=tk.W)
        ttk.Radiobutton(auto_frame, text="Body Detection", variable=self.crop_mode, value="body").pack(anchor=tk.W)
        ttk.Radiobutton(auto_frame, text="Center Crop", variable=self.crop_mode, value="center").pack(anchor=tk.W)
        
        # Crop size settings
        size_frame = ttk.Frame(auto_frame)
        size_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(size_frame, text="Width:").grid(row=0, column=0, sticky=tk.W)
        self.width_var = tk.StringVar(value="700")
        ttk.Entry(size_frame, textvariable=self.width_var, width=8).grid(row=0, column=1, padx=5)
        
        ttk.Label(size_frame, text="Height:").grid(row=1, column=0, sticky=tk.W)
        self.height_var = tk.StringVar(value="700")
        ttk.Entry(size_frame, textvariable=self.height_var, width=8).grid(row=1, column=1, padx=5)
        
        ttk.Button(auto_frame, text="Auto Crop All", command=self.auto_crop_all).pack(fill=tk.X, pady=5)
        
        # Auto crop info
        ttk.Label(auto_frame, text="Auto crops resize to specified", font=('TkDefaultFont', 8)).pack(pady=(5,0))
        ttk.Label(auto_frame, text="width/height dimensions", font=('TkDefaultFont', 8)).pack()
        
        # Output settings
        output_frame = ttk.LabelFrame(left_frame, text="Output Settings", padding=10)
        output_frame.pack(fill=tk.X, pady=(0, 10))


        ttk.Button(output_frame, text="Select Background Color", command=self.select_background_color).pack(fill=tk.X, pady=2)

        # ✅ Add this line
        self.apply_bg_color = tk.BooleanVar(value=False)

        # Background color toggle
        ttk.Checkbutton(output_frame, text="Apply Background Color", variable=self.apply_bg_color).pack(anchor=tk.W, pady=2)

        # Background color picker
        ttk.Button(output_frame, text="Select Background Color", command=self.select_background_color).pack(fill=tk.X, pady=2)

        # Color preview
        self.bg_color_var = tk.StringVar(value="#FFFFFF")
        self.color_preview = tk.Label(output_frame, text="      ", background=self.bg_color_var.get(), borderwidth=1, relief="solid")
        self.color_preview.pack(anchor=tk.W, pady=2)

        # Display hex value
        ttk.Label(output_frame, textvariable=self.bg_color_var).pack(anchor=tk.W)


        
        ttk.Button(output_frame, text="Select Output Folder", command=self.select_output_folder).pack(fill=tk.X, pady=2)
        
        self.output_folder = tk.StringVar(value="./cropped_images")
        ttk.Label(output_frame, text="Output:").pack(anchor=tk.W)
        ttk.Label(output_frame, textvariable=self.output_folder, wraplength=200).pack(anchor=tk.W)
        
        # Progress bar
        self.progress = ttk.Progressbar(left_frame, mode='determinate')
        self.progress.pack(fill=tk.X, pady=10)
        
        # Right panel - Image display
        right_frame = ttk.Frame(main_frame)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Canvas for image display
        self.canvas = tk.Canvas(right_frame, bg='white', cursor='cross')
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Bind mouse events for manual cropping
        self.canvas.bind("<Button-1>", self.start_crop)
        self.canvas.bind("<B1-Motion>", self.draw_crop)
        self.canvas.bind("<ButtonRelease-1>", self.end_crop)
        
        # Status bar
        self.status_label = ttk.Label(self.root, text="Ready", relief=tk.SUNKEN)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X)


    def select_background_color(self):
        color_code = colorchooser.askcolor(title="Choose Background Color")[1]
        if color_code:
            self.bg_color_var.set(color_code)
            self.color_preview.config(background=color_code)

    
    def hex_to_bgr(self, hex_color):
        h = hex_color.lstrip('#')
        return tuple(int(h[i:i+2], 16) for i in (4, 2, 0))  # BGR order




        
    def select_images(self):
        try:
            files = filedialog.askopenfilenames(
                title="Select Images",
                filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif")]
            )
            if files:  # Only proceed if files were actually selected
                self.selected_images = list(files)
                self.update_image_count()
                self.current_image_index = 0
                self.load_current_image()
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting images: {str(e)}")
            
    def select_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select Image Folder")
            if folder:
                image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
                self.selected_images = []
                for file_path in Path(folder).rglob('*'):
                    if file_path.suffix.lower() in image_extensions:
                        self.selected_images.append(str(file_path))
                self.update_image_count()
                if self.selected_images:
                    self.current_image_index = 0
                    self.load_current_image()
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting folder: {str(e)}")
                
    def select_output_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select Output Folder")
            if folder:
                self.output_folder.set(folder)
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting output folder: {str(e)}")
            
    def update_image_count(self):
        count = len(self.selected_images)
        self.image_count_label.config(text=f"{count} images selected")
        
    def load_current_image(self):
        if not self.selected_images:
            return
            
        try:
            image_path = self.selected_images[self.current_image_index]




            #load with pil first to get DPI
            pil_img = Image.open(image_path)
            self.current_image = cv2.imread(image_path)
            
            if self.current_image is None:
                messagebox.showerror("Error", f"Could not load image: {image_path}")
                return
                
            # Update current image label
            filename = os.path.basename(image_path)
            self.current_image_label.config(text=f"{self.current_image_index + 1}/{len(self.selected_images)}: {filename}")
            
            # Display image
            self.display_image_on_canvas()
        except Exception as e:
            messagebox.showerror("Error", f"Error loading image: {str(e)}")
        
    def display_image_on_canvas(self):
        if self.current_image is None:
            return
            
        try:
            # Check if root window still exists
            if not self.root.winfo_exists():
                return
                
            # Convert BGR to RGB
            rgb_image = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
            
            # Calculate scale factor to fit canvas
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            if canvas_width <= 1 or canvas_height <= 1:
                self.root.after(100, self.display_image_on_canvas)
                return
                
            img_height, img_width = rgb_image.shape[:2]
            
            scale_x = canvas_width / img_width
            scale_y = canvas_height / img_height
            self.scale_factor = min(scale_x, scale_y, 1.0)
            
            new_width = int(img_width * self.scale_factor)
            new_height = int(img_height * self.scale_factor)
            
            # Resize image
            resized_image = cv2.resize(rgb_image, (new_width, new_height))
            
            # Convert to PIL Image
            pil_image = Image.fromarray(resized_image)
            
            # Create PhotoImage with explicit master reference
            self.display_image = ImageTk.PhotoImage(pil_image, master=self.root)
            
            # Clear canvas and display image
            self.canvas.delete("all")
            self.canvas.create_image(canvas_width//2, canvas_height//2, image=self.display_image)
            
        except Exception as e:
            print(f"Error displaying image: {str(e)}")
            # Try to show error message if root still exists
            try:
                if self.root.winfo_exists():
                    messagebox.showerror("Error", f"Error displaying image: {str(e)}")
            except:
                pass
        
    def start_crop(self, event):
        self.is_cropping = True
        self.start_x = event.x
        self.start_y = event.y
        if self.rect_id:
            self.canvas.delete(self.rect_id)
            
    def draw_crop(self, event):
        if self.is_cropping:
            if self.rect_id:
                self.canvas.delete(self.rect_id)
            self.rect_id = self.canvas.create_rectangle(
                self.start_x, self.start_y, event.x, event.y,
                outline='red', width=2
            )
            
    def end_crop(self, event):
        if self.is_cropping:
            self.is_cropping = False
            
            # Calculate crop coordinates in original image
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            # Get image position on canvas
            img_height, img_width = self.current_image.shape[:2]
            scaled_width = int(img_width * self.scale_factor)
            scaled_height = int(img_height * self.scale_factor)
            
            img_x = (canvas_width - scaled_width) // 2
            img_y = (canvas_height - scaled_height) // 2
            
            # Convert canvas coordinates to image coordinates
            x1 = max(0, int((self.start_x - img_x) / self.scale_factor))
            y1 = max(0, int((self.start_y - img_y) / self.scale_factor))
            x2 = min(img_width, int((event.x - img_x) / self.scale_factor))
            y2 = min(img_height, int((event.y - img_y) / self.scale_factor))
            
            if x2 > x1 and y2 > y1:
                self.crop_coords = (x1, y1, x2, y2)
            else:
                self.crop_coords = None
                if self.rect_id:
                    self.canvas.delete(self.rect_id)
                    self.rect_id = None
                

                
    def reset_crop(self):
        if self.rect_id:
            self.canvas.delete(self.rect_id)
            self.rect_id = None
        self.crop_coords = None



        
    def crop_current_manual(self):
        if self.current_image is not None and self.crop_coords:
            try:
                # Get the crop coordinates in original image dimensions
                x1, y1, x2, y2 = self.crop_coords
                
                # Ensure coordinates are within image bounds
                img_height, img_width = self.current_image.shape[:2]
                x1 = max(0, min(x1, img_width - 1))
                y1 = max(0, min(y1, img_height - 1))
                x2 = max(0, min(x2, img_width))
                y2 = max(0, min(y2, img_height))
                
                # Ensure valid dimensions
                if x2 > x1 and y2 > y1:
                    cropped = self.current_image[y1:y2, x1:x2]
                    self.save_cropped_image_manual(cropped, self.current_image_index)
                    messagebox.showinfo("Success", "Image cropped and saved!")
                else:
                    messagebox.showwarning("Warning", "Invalid crop area selected!")
            except Exception as e:
                messagebox.showerror("Error", f"Error cropping image: {str(e)}")
        else:
            messagebox.showwarning("Warning", "Please select a crop area first!")


            
    def prev_image(self):
        if self.selected_images and self.current_image_index > 0:
            try:
                self.current_image_index -= 1
                self.load_current_image()
                self.reset_crop()
            except Exception as e:
                messagebox.showerror("Error", f"Error loading previous image: {str(e)}")
            
    def next_image(self):
        if self.selected_images and self.current_image_index < len(self.selected_images) - 1:
            try:
                self.current_image_index += 1
                self.load_current_image()
                self.reset_crop()
            except Exception as e:
                messagebox.showerror("Error", f"Error loading next image: {str(e)}")
            
    def auto_crop_all(self):
        if not self.selected_images:
            messagebox.showwarning("Warning", "Please select images first!")
            return
            
        # Create output directory
        output_dir = self.output_folder.get()
        os.makedirs(output_dir, exist_ok=True)
        
        # Start processing in a separate thread
        thread = threading.Thread(target=self.process_images_thread)
        thread.daemon = True
        thread.start()
        
    def process_images_thread(self):
        total_images = len(self.selected_images)
        self.progress['maximum'] = total_images
        
        try:
            target_width = int(self.width_var.get())
            target_height = int(self.height_var.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid width and height values!")
            return
            
        success_count = 0
        
        for i, image_path in enumerate(self.selected_images):
            try:
                # Update status
                self.root.after(0, lambda: self.status_label.config(text=f"Processing {i+1}/{total_images}"))
                
                # Load image
                image = cv2.imread(image_path)
                if image is None:
                    continue
                    
                # Apply auto cropping based on mode
                mode = self.crop_mode.get()
                cropped_image = self.auto_crop_image(image, mode, target_width, target_height)
                
                if cropped_image is not None:
                    self.save_cropped_image(cropped_image, i)
                    success_count += 1
                    
                # Update progress
                self.root.after(0, lambda: self.progress.config(value=i+1))
                
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
                continue
                
        # Show completion message
        self.root.after(0, lambda: messagebox.showinfo("Complete", f"Processed {success_count}/{total_images} images successfully!"))
        self.root.after(0, lambda: self.status_label.config(text="Ready"))
        self.root.after(0, lambda: self.progress.config(value=0))
        
    def auto_crop_image(self, image, mode, target_width, target_height):
        height, width = image.shape[:2]
        
        if mode == "face":
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
            print(f"Detected {len(faces)} faces.")


            if len(faces) > 0:
                largest_face = max(faces, key=lambda x: x[2] * x[3])
                x, y, w, h = largest_face

                padding = max(w, h) // 2
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(width, x + w + padding)
                y2 = min(height, y + h + padding)

                if self.apply_bg_color.get():
                    # Apply GrabCut to image
                    mask = np.zeros(image.shape[:2], np.uint8)
                    bgdModel = np.zeros((1, 65), np.float64)
                    fgdModel = np.zeros((1, 65), np.float64)
                    rect = (x1, y1, x2 - x1, y2 - y1)

                    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
                    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

                    person = image * mask2[:, :, np.newaxis]

                    bgr_color = self.hex_to_bgr(self.bg_color_var.get())
                    background = np.full(image.shape, bgr_color, dtype=np.uint8)

                    result = np.where(mask2[:, :, np.newaxis] == 0, background, person)

                    cropped = result[y1:y2, x1:x2]

                else:
                    cropped = image[y1:y2, x1:x2]

                # Apply Gaussian blur
                cropped = cv2.GaussianBlur(cropped, (3, 3), 0.5)

                # Resize to target size
                cropped = cv2.resize(cropped, (target_width, target_height), interpolation=cv2.INTER_AREA)

                # Apply CLAHE enhancement
                lab = cv2.cvtColor(cropped, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8, 8))
                cl = clahe.apply(l)
                merged_lab = cv2.merge((cl, a, b))
                enhanced = cv2.cvtColor(merged_lab, cv2.COLOR_LAB2BGR)

                return enhanced


                
        elif mode == "body":
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            bodies = self.body_cascade.detectMultiScale(gray, 1.1, 4)

            if len(bodies) > 0:
                largest_body = max(bodies, key=lambda x: x[2] * x[3])
                x, y, w, h = largest_body

                padding = max(w, h) // 4
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(width, x + w + padding)
                y2 = min(height, y + h + padding)

                if self.apply_bg_color.get():
                    # Apply GrabCut to segment body from background
                    mask = np.zeros(image.shape[:2], np.uint8)
                    bgdModel = np.zeros((1, 65), np.float64)
                    fgdModel = np.zeros((1, 65), np.float64)
                    rect = (x1, y1, x2 - x1, y2 - y1)

                    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
                    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

                    person = image * mask2[:, :, np.newaxis]

                    bgr_color = self.hex_to_bgr(self.bg_color_var.get())
                    background = np.full(image.shape, bgr_color, dtype=np.uint8)

                    result = np.where(mask2[:, :, np.newaxis] == 0, background, person)

                    cropped = result[y1:y2, x1:x2]

                else:
                    cropped = image[y1:y2, x1:x2]

                # ✅ Common processing for both paths

                # Apply Gaussian blur
                cropped = cv2.GaussianBlur(cropped, (3, 3), 0.5)

                # Resize to target size
                cropped = cv2.resize(cropped, (target_width, target_height), interpolation=cv2.INTER_AREA)

                # Apply CLAHE enhancement
                lab = cv2.cvtColor(cropped, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=0.5, tileGridSize=(8, 8))
                cl = clahe.apply(l)
                merged_lab = cv2.merge((cl, a, b))
                enhanced = cv2.cvtColor(merged_lab, cv2.COLOR_LAB2BGR)

                return enhanced



                
        # Default to center crop
        aspect_ratio = target_width / target_height
        
        if width / height > aspect_ratio:
            # Image is wider - crop width
            new_width = int(height * aspect_ratio)
            x_offset = (width - new_width) // 2
            cropped = image[:, x_offset:x_offset + new_width]
        else:
            # Image is taller - crop height
            new_height = int(width / aspect_ratio)
            y_offset = (height - new_height) // 2
            cropped = image[y_offset:y_offset + new_height, :]
            
        return cv2.resize(cropped, (target_width, target_height))
        
    def save_cropped_image_manual(self, cropped_image, index):
        """Save manually cropped image without resizing - preserves exact crop dimensions"""
        try:
            output_dir = self.output_folder.get()
            os.makedirs(output_dir, exist_ok=True)
            
            original_path = self.selected_images[index]
            filename = os.path.basename(original_path)
            name, ext = os.path.splitext(filename)
            
            # Add manual crop suffix to distinguish from auto-cropped images
            output_path = os.path.join(output_dir, f"{name}_manual_crop{ext}")
            success = cv2.imwrite(output_path, cropped_image)
            
            if not success:
                raise Exception(f"Failed to save image to {output_path}")
                
        except Exception as e:
            print(f"Error saving manually cropped image: {str(e)}")
            raise e
    
    def save_cropped_image(self, cropped_image, index):
        """Save automatically cropped image with resizing applied"""
        try:
            output_dir = self.output_folder.get()
            os.makedirs(output_dir, exist_ok=True)
            
            original_path = self.selected_images[index]
            filename = os.path.basename(original_path)
            name, ext = os.path.splitext(filename)
            
            # Add auto crop suffix to distinguish from manually cropped images
            output_path = os.path.join(output_dir, f"{name}_auto_crop{ext}")
            ext = os.path.splitext(output_path)[1].lower()
            if ext in ['.jpg', '.jpeg']:
                success = cv2.imwrite(output_path, cropped_image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
            else:
                success = cv2.imwrite(output_path, cropped_image)

            
            if not success:
                raise Exception(f"Failed to save image to {output_path}")
                
        except Exception as e:
            print(f"Error saving cropped image: {str(e)}")
            raise e

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = ImageCropper(root)
        root.mainloop()
    except Exception as e:
        print(f"Error starting application: {str(e)}")
        import traceback
        traceback.print_exc()

Detected 1 faces.
Detected 2 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 4 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 2 faces.
Detected 1 faces.
Detected 1 faces.
Detected 1 faces.
Detected 2 faces.
Detected 1 faces.
Detected 1 faces.


In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, colorchooser
import cv2
import numpy as np
from PIL import Image, ImageTk
import os
import threading
from pathlib import Path
from rembg import remove

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Bulk Image Cropper")
        self.root.geometry("1200x800")
        
        # Variables
        self.selected_images = []
        self.current_image_index = 0
        self.current_image = None
        self.display_image = None
        self.crop_coords = None
        self.is_cropping = False
        self.start_x = self.start_y = 0
        self.rect_id = None
        self.scale_factor = 1.0
        
        # Load face and body cascades
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
        
        # Background color toggle
        self.apply_bg_color = tk.BooleanVar(value=False)
        
        # Edge quality control
        self.edge_smoothness = tk.DoubleVar(value=10.0)
        
        # Advanced processing toggle
        self.use_advanced_processing = tk.BooleanVar(value=True)
        
        self.setup_ui()
        
    def setup_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left panel - Controls
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # File selection
        file_frame = ttk.LabelFrame(left_frame, text="Image Selection", padding=10)
        file_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(file_frame, text="Select Images", command=self.select_images).pack(fill=tk.X, pady=2)
        ttk.Button(file_frame, text="Select Folder", command=self.select_folder).pack(fill=tk.X, pady=2)
        
        self.image_count_label = ttk.Label(file_frame, text="No images selected")
        self.image_count_label.pack(pady=5)
        
        # Manual cropping controls
        manual_frame = ttk.LabelFrame(left_frame, text="Manual Cropping", padding=10)
        manual_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(manual_frame, text="Previous Image", command=self.prev_image).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Next Image", command=self.next_image).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Crop Current", command=self.crop_current_manual).pack(fill=tk.X, pady=2)
        ttk.Button(manual_frame, text="Reset Crop", command=self.reset_crop).pack(fill=tk.X, pady=2)
        
        # Manual crop info
        ttk.Label(manual_frame, text="Manual crops preserve exact", font=('TkDefaultFont', 8)).pack(pady=(5,0))
        ttk.Label(manual_frame, text="bounding box dimensions", font=('TkDefaultFont', 8)).pack()
        
        self.current_image_label = ttk.Label(manual_frame, text="No image loaded")
        self.current_image_label.pack(pady=5)
        
        # Automatic cropping controls
        auto_frame = ttk.LabelFrame(left_frame, text="Automatic Cropping", padding=10)
        auto_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.crop_mode = tk.StringVar(value="face")
        ttk.Radiobutton(auto_frame, text="Face Detection", variable=self.crop_mode, value="face").pack(anchor=tk.W)
        ttk.Radiobutton(auto_frame, text="Body Detection", variable=self.crop_mode, value="body").pack(anchor=tk.W)
        ttk.Radiobutton(auto_frame, text="Center Crop", variable=self.crop_mode, value="center").pack(anchor=tk.W)
        
        # Crop size settings
        size_frame = ttk.Frame(auto_frame)
        size_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(size_frame, text="Width:").grid(row=0, column=0, sticky=tk.W)
        self.width_var = tk.StringVar(value="300")
        ttk.Entry(size_frame, textvariable=self.width_var, width=8).grid(row=0, column=1, padx=5)
        
        ttk.Label(size_frame, text="Height:").grid(row=1, column=0, sticky=tk.W)
        self.height_var = tk.StringVar(value="300")
        ttk.Entry(size_frame, textvariable=self.height_var, width=8).grid(row=1, column=1, padx=5)
        
        ttk.Button(auto_frame, text="Auto Crop All", command=self.auto_crop_all).pack(fill=tk.X, pady=5)
        
        # Auto crop info
        ttk.Label(auto_frame, text="Auto crops resize to specified", font=('TkDefaultFont', 8)).pack(pady=(5,0))
        ttk.Label(auto_frame, text="width/height dimensions", font=('TkDefaultFont', 8)).pack()
        
        # Output settings
        output_frame = ttk.LabelFrame(left_frame, text="Output Settings", padding=10)
        output_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Background color toggle
        ttk.Checkbutton(output_frame, text="Apply Background Color", variable=self.apply_bg_color).pack(anchor=tk.W, pady=2)
        
        # Background color picker
        ttk.Button(output_frame, text="Select Background Color", command=self.select_background_color).pack(fill=tk.X, pady=2)
        
        # Color preview
        self.bg_color_var = tk.StringVar(value="#FFFFFF")
        self.color_preview = tk.Label(output_frame, text="      ", background=self.bg_color_var.get(), borderwidth=1, relief="solid")
        self.color_preview.pack(anchor=tk.W, pady=2)
        
        # Display hex value
        ttk.Label(output_frame, textvariable=self.bg_color_var).pack(anchor=tk.W)
        
        # Edge quality settings
        edge_frame = ttk.LabelFrame(output_frame, text="Edge Quality", padding=5)
        edge_frame.pack(fill=tk.X, pady=5)
        
        ttk.Label(edge_frame, text="Edge Smoothness:").pack(anchor=tk.W)
        edge_scale = ttk.Scale(edge_frame, from_=5.0, to=20.0, variable=self.edge_smoothness, orient=tk.HORIZONTAL)
        edge_scale.pack(fill=tk.X, pady=2)
        
        ttk.Label(edge_frame, text="Higher = softer edges", font=('TkDefaultFont', 8)).pack(anchor=tk.W)
        
        # Advanced processing toggle
        ttk.Checkbutton(edge_frame, text="Ultra-Natural Edge Processing", 
                       variable=self.use_advanced_processing).pack(anchor=tk.W, pady=5)
        ttk.Label(edge_frame, text="Uses advanced algorithms for", font=('TkDefaultFont', 8)).pack(anchor=tk.W)
        ttk.Label(edge_frame, text="professional-quality results", font=('TkDefaultFont', 8)).pack(anchor=tk.W)
        
        ttk.Button(output_frame, text="Select Output Folder", command=self.select_output_folder).pack(fill=tk.X, pady=2)
        
        self.output_folder = tk.StringVar(value="./cropped_images")
        ttk.Label(output_frame, text="Output:").pack(anchor=tk.W)
        ttk.Label(output_frame, textvariable=self.output_folder, wraplength=200).pack(anchor=tk.W)
        
        # Progress bar
        self.progress = ttk.Progressbar(left_frame, mode='determinate')
        self.progress.pack(fill=tk.X, pady=10)
        
        # Right panel - Image display
        right_frame = ttk.Frame(main_frame)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Canvas for image display
        self.canvas = tk.Canvas(right_frame, bg='white', cursor='cross')
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Bind mouse events for manual cropping
        self.canvas.bind("<Button-1>", self.start_crop)
        self.canvas.bind("<B1-Motion>", self.draw_crop)
        self.canvas.bind("<ButtonRelease-1>", self.end_crop)
        
        # Status bar
        self.status_label = ttk.Label(self.root, text="Ready", relief=tk.SUNKEN)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X)

    def select_background_color(self):
        color_code = colorchooser.askcolor(title="Choose Background Color")[1]
        if color_code:
            self.bg_color_var.set(color_code)
            self.color_preview.config(background=color_code)
    
    def hex_to_bgr(self, hex_color):
        h = hex_color.lstrip('#')
        return tuple(int(h[i:i+2], 16) for i in (4, 2, 0))  # BGR order

    def select_images(self):
        try:
            files = filedialog.askopenfilenames(
                title="Select Images",
                filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif")]
            )
            if files:
                self.selected_images = list(files)
                self.update_image_count()
                self.current_image_index = 0
                self.load_current_image()
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting images: {str(e)}")

    def select_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select Image Folder")
            if folder:
                image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
                self.selected_images = []
                for file_path in Path(folder).rglob('*'):
                    if file_path.suffix.lower() in image_extensions:
                        self.selected_images.append(str(file_path))
                self.update_image_count()
                if self.selected_images:
                    self.current_image_index = 0
                    self.load_current_image()
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting folder: {str(e)}")

    def select_output_folder(self):
        try:
            folder = filedialog.askdirectory(title="Select Output Folder")
            if folder:
                self.output_folder.set(folder)
        except Exception as e:
            messagebox.showerror("Error", f"Error selecting output folder: {str(e)}")

    def update_image_count(self):
        count = len(self.selected_images)
        self.image_count_label.config(text=f"{count} images selected")

    def load_current_image(self):
        if not self.selected_images:
            return
            
        try:
            image_path = self.selected_images[self.current_image_index]
            pil_img = Image.open(image_path)
            self.current_image = cv2.imread(image_path)
            
            if self.current_image is None:
                messagebox.showerror("Error", f"Could not load image: {image_path}")
                return
                
            # Update current image label
            filename = os.path.basename(image_path)
            self.current_image_label.config(text=f"{self.current_image_index + 1}/{len(self.selected_images)}: {filename}")
            
            # Display image
            self.display_image_on_canvas()
        except Exception as e:
            messagebox.showerror("Error", f"Error loading image: {str(e)}")

    def display_image_on_canvas(self):
        if self.current_image is None:
            return
            
        try:
            if not self.root.winfo_exists():
                return
                
            rgb_image = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
            
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            if canvas_width <= 1 or canvas_height <= 1:
                self.root.after(100, self.display_image_on_canvas)
                return
                
            img_height, img_width = rgb_image.shape[:2]
            
            scale_x = canvas_width / img_width
            scale_y = canvas_height / img_height
            self.scale_factor = min(scale_x, scale_y, 1.0)
            
            new_width = int(img_width * self.scale_factor)
            new_height = int(img_height * self.scale_factor)
            
            resized_image = cv2.resize(rgb_image, (new_width, new_height))
            
            pil_image = Image.fromarray(resized_image)
            self.display_image = ImageTk.PhotoImage(pil_image, master=self.root)
            
            self.canvas.delete("all")
            self.canvas.create_image(canvas_width//2, canvas_height//2, image=self.display_image)
            
        except Exception as e:
            print(f"Error displaying image: {str(e)}")
            try:
                if self.root.winfo_exists():
                    messagebox.showerror("Error", f"Error displaying image: {str(e)}")
            except:
                pass

    def start_crop(self, event):
        self.is_cropping = True
        self.start_x = event.x
        self.start_y = event.y
        if self.rect_id:
            self.canvas.delete(self.rect_id)

    def draw_crop(self, event):
        if self.is_cropping:
            if self.rect_id:
                self.canvas.delete(self.rect_id)
            self.rect_id = self.canvas.create_rectangle(
                self.start_x, self.start_y, event.x, event.y,
                outline='red', width=2
            )

    def end_crop(self, event):
        if self.is_cropping:
            self.is_cropping = False
            
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            img_height, img_width = self.current_image.shape[:2]
            scaled_width = int(img_width * self.scale_factor)
            scaled_height = int(img_height * self.scale_factor)
            
            img_x = (canvas_width - scaled_width) // 2
            img_y = (canvas_height - scaled_height) // 2
            
            x1 = max(0, int((self.start_x - img_x) / self.scale_factor))
            y1 = max(0, int((self.start_y - img_y) / self.scale_factor))
            x2 = min(img_width, int((event.x - img_x) / self.scale_factor))
            y2 = min(img_height, int((event.y - img_y) / self.scale_factor))
            
            if x2 > x1 and y2 > y1:
                self.crop_coords = (x1, y1, x2, y2)
            else:
                self.crop_coords = None
                if self.rect_id:
                    self.canvas.delete(self.rect_id)
                    self.rect_id = None

    def reset_crop(self):
        if self.rect_id:
            self.canvas.delete(self.rect_id)
            self.rect_id = None
        self.crop_coords = None

    def crop_current_manual(self):
        if self.current_image is not None and self.crop_coords:
            try:
                x1, y1, x2, y2 = self.crop_coords
                
                img_height, img_width = self.current_image.shape[:2]
                x1 = max(0, min(x1, img_width - 1))
                y1 = max(0, min(y1, img_height - 1))
                x2 = max(0, min(x2, img_width))
                y2 = max(0, min(y2, img_height))
                
                if x2 > x1 and y2 > y1:
                    cropped = self.current_image[y1:y2, x1:x2]
                    if self.apply_bg_color.get():
                        rgb_image = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                        pil_image = Image.fromarray(rgb_image)
                        # Ensure RGB format before converting to RGBA
                        if pil_image.mode != 'RGB':
                            pil_image = pil_image.convert('RGB')
                        pil_image_rgba = pil_image.convert("RGBA")
                        processed = self.apply_background_replacement(pil_image_rgba)
                        cropped = cv2.cvtColor(processed, cv2.COLOR_RGB2BGR)
                    self.save_cropped_image_manual(cropped, self.current_image_index)
                    messagebox.showinfo("Success", "Image cropped and saved!")
                else:
                    messagebox.showwarning("Warning", "Invalid crop area selected!")
            except Exception as e:
                messagebox.showerror("Error", f"Error cropping image: {str(e)}")
        else:
            messagebox.showwarning("Warning", "Please select a crop area first!")

    def prev_image(self):
        if self.selected_images and self.current_image_index > 0:
            try:
                self.current_image_index -= 1
                self.load_current_image()
                self.reset_crop()
            except Exception as e:
                messagebox.showerror("Error", f"Error loading previous image: {str(e)}")

    def next_image(self):
        if self.selected_images and self.current_image_index < len(self.selected_images) - 1:
            try:
                self.current_image_index += 1
                self.load_current_image()
                self.reset_crop()
            except Exception as e:
                messagebox.showerror("Error", f"Error loading next image: {str(e)}")

    def refine_alpha_mask(self, alpha):
        if alpha.ndim == 3:
            alpha_2d = alpha.squeeze()
        else:
            alpha_2d = alpha
            
        alpha_uint8 = (alpha_2d * 255).astype(np.uint8)
        
        # Step 1: Advanced edge-preserving smoothing
        try:
            # Use multiple iterations of bilateral filtering for better edge preservation
            alpha_smooth = alpha_uint8.copy()
            for _ in range(2):
                alpha_smooth = cv2.bilateralFilter(alpha_smooth, 5, 50, 50)
        except Exception as e:
            print(f"Bilateral filter failed: {e}")
            alpha_smooth = cv2.GaussianBlur(alpha_uint8, (3, 3), 0)
        
        # Step 2: Create trimap for better matting
        # Define certain foreground, certain background, and uncertain regions
        sure_fg = cv2.threshold(alpha_smooth, 240, 255, cv2.THRESH_BINARY)[1]
        sure_bg = cv2.threshold(alpha_smooth, 10, 255, cv2.THRESH_BINARY_INV)[1]
        uncertain = cv2.subtract(255 - sure_bg, sure_fg)
        
        # Step 3: Advanced alpha matting using guided filter approach
        alpha_float = alpha_smooth.astype(np.float32) / 255.0
        
        # Create a softer transition in uncertain areas
        if np.any(uncertain > 0):
            # Apply guided filter-like smoothing in uncertain regions
            uncertain_mask = uncertain > 0
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
            
            # Create distance-based falloff in uncertain areas
            dist_transform = cv2.distanceTransform(sure_fg, cv2.DIST_L2, 5)
            smoothness = self.edge_smoothness.get()
            
            # Normalize distance transform
            if dist_transform.max() > 0:
                dist_normalized = dist_transform / dist_transform.max()
                # Apply smoothness parameter
                dist_falloff = np.power(dist_normalized, 1.0 / (smoothness / 10.0))
                
                # Blend in uncertain regions
                alpha_float = np.where(uncertain_mask, 
                                     dist_falloff * alpha_float + (1 - dist_falloff) * 0.1,
                                     alpha_float)
        
        # Step 4: Multi-scale edge refinement
        # Create multiple scales of the alpha mask and blend them
        scales = [1, 2, 4]  # Different blur scales
        blended_alpha = np.zeros_like(alpha_float)
        total_weight = 0
        
        for scale in scales:
            if scale == 1:
                scale_alpha = alpha_float
                weight = 0.6  # Highest weight for original scale
            else:
                kernel_size = scale * 2 + 1
                scale_alpha = cv2.GaussianBlur(alpha_float, (kernel_size, kernel_size), scale * 0.5)
                weight = 0.2  # Lower weight for blurred scales
            
            blended_alpha += scale_alpha * weight
            total_weight += weight
        
        blended_alpha /= total_weight
        
        # Step 5: Final edge softening with adaptive blur
        # Apply stronger blur to areas with high gradient (edges)
        gradient = cv2.Sobel(alpha_uint8, cv2.CV_64F, 1, 1, ksize=3)
        gradient_mag = np.sqrt(gradient**2)
        gradient_normalized = gradient_mag / (gradient_mag.max() + 1e-8)
        
        # Create adaptive blur kernel based on gradient
        final_alpha = blended_alpha.copy()
        for i in range(3):  # Multiple passes for smoother result
            adaptive_blur = cv2.GaussianBlur(final_alpha, (5, 5), 1.0)
            # Blend more blur in high-gradient areas
            blend_factor = gradient_normalized * 0.7
            final_alpha = final_alpha * (1 - blend_factor) + adaptive_blur * blend_factor
        
        return np.clip(final_alpha, 0, 1)

    def grabcut_refine_mask(self, image, initial_mask):
        """Use GrabCut to refine the mask for better edges"""
        try:
            # Convert image to BGR for OpenCV
            img_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            
            # Create mask for GrabCut (0: background, 1: foreground, 2: probable background, 3: probable foreground)
            mask = np.zeros(initial_mask.shape[:2], np.uint8)
            
            # Set initial mask based on alpha values
            mask[initial_mask > 0.8] = 1  # Sure foreground
            mask[initial_mask < 0.2] = 0  # Sure background  
            mask[(initial_mask >= 0.2) & (initial_mask <= 0.8)] = 3  # Probable foreground
            
            # Initialize models
            bgd_model = np.zeros((1, 65), np.float64)
            fgd_model = np.zeros((1, 65), np.float64)
            
            # Apply GrabCut
            cv2.grabCut(img_bgr, mask, None, bgd_model, fgd_model, 3, cv2.GC_INIT_WITH_MASK)
            
            # Create final mask
            refined_mask = np.where((mask == 2) | (mask == 0), 0, 1).astype(np.float32)
            
            # Smooth the refined mask
            refined_mask = cv2.GaussianBlur(refined_mask, (3, 3), 1)
            
            return refined_mask
            
        except Exception as e:
            print(f"GrabCut refinement failed: {e}")
            return initial_mask.squeeze() if initial_mask.ndim == 3 else initial_mask

    def preprocess_image_for_rembg(self, image):
        """Preprocess image to improve rembg performance"""
        # Convert PIL to numpy array and ensure it's RGB format
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        img_array = np.array(image)
        
        # Ensure the image is uint8 format
        if img_array.dtype != np.uint8:
            img_array = img_array.astype(np.uint8)
        
        # Ensure the image has 3 channels (RGB)
        if len(img_array.shape) != 3 or img_array.shape[2] != 3:
            return image  # Return original if format is unexpected
        
        try:
            # Apply gentle noise reduction while preserving edges
            denoised = cv2.bilateralFilter(img_array, 9, 75, 75)
            
            # Enhance contrast slightly to help with edge detection
            lab = cv2.cvtColor(denoised, cv2.COLOR_RGB2LAB)
            l_channel = lab[:, :, 0]
            
            # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to L channel
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            l_channel = clahe.apply(l_channel)
            
            lab[:, :, 0] = l_channel
            enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
            
            return Image.fromarray(enhanced)
        except Exception as e:
            print(f"Preprocessing failed, using original image: {e}")
            return image

    def color_spill_suppression(self, image, alpha, background_color):
        """Remove color spill from original background"""
        img_float = image.astype(np.float32) / 255.0
        bg_color = np.array(background_color[:3]) / 255.0  # RGB
        
        # Detect areas with low alpha (edges) where spill is likely
        edge_mask = (alpha < 0.95) & (alpha > 0.05)
        
        if not np.any(edge_mask):
            return image
        
        # For each channel, reduce the influence of the original background
        for c in range(3):
            # Calculate the deviation from the background color
            deviation = np.abs(img_float[:, :, c] - bg_color[c])
            # Apply spill suppression in edge areas
            spill_factor = np.clip(deviation * 2, 0, 1)
            img_float[edge_mask, c] = img_float[edge_mask, c] * spill_factor[edge_mask] + \
                                     bg_color[c] * (1 - spill_factor[edge_mask])
        
        return (img_float * 255).astype(np.uint8)

    def apply_background_replacement(self, image):
        # Preprocess the image for better rembg results
        preprocessed_image = self.preprocess_image_for_rembg(image)
        
        # Remove background using the preprocessed image
        removed = remove(preprocessed_image)
        fg = np.array(removed).astype(np.float32) / 255.0
        alpha_raw = fg[:, :, 3:4]
        
        # Get refined alpha mask
        alpha_refined = self.refine_alpha_mask(alpha_raw)
        
        # Further refine with GrabCut for ultra-precise edges (if advanced processing enabled)
        if self.use_advanced_processing.get():
            alpha_refined = self.grabcut_refine_mask(image, alpha_refined)
        
        # Prepare original foreground (use original image, not preprocessed)
        original_fg = np.array(image).astype(np.float32) / 255.0
        if original_fg.shape[2] == 4:  # If RGBA, take only RGB
            original_fg = original_fg[:, :, :3]
        
        # Create background with selected color
        bg_color_rgb = self.hex_to_bgr(self.bg_color_var.get())
        bg_color_rgb = (bg_color_rgb[2], bg_color_rgb[1], bg_color_rgb[0])  # Convert BGR to RGB
        
        # Step 1: Color spill suppression
        original_fg_uint8 = (original_fg * 255).astype(np.uint8)
        spill_corrected = self.color_spill_suppression(original_fg_uint8, alpha_refined, bg_color_rgb)
        spill_corrected_float = spill_corrected.astype(np.float32) / 255.0
        
        # Step 2: Edge color harmonization
        # Blend edge colors with background color for natural look
        alpha_2d = alpha_refined.squeeze() if alpha_refined.ndim == 3 else alpha_refined
        edge_mask = (alpha_2d > 0.1) & (alpha_2d < 0.9)
        
        if np.any(edge_mask):
            bg_influence = (1 - alpha_2d[edge_mask]) * 0.3  # 30% background influence on edges
            for c in range(3):
                spill_corrected_float[edge_mask, c] = \
                    spill_corrected_float[edge_mask, c] * (1 - bg_influence) + \
                    (bg_color_rgb[c] / 255.0) * bg_influence
        
        # Step 3: Create sophisticated background
        bg = np.full((original_fg.shape[0], original_fg.shape[1], 3), 
                     [bg_color_rgb[0]/255.0, bg_color_rgb[1]/255.0, bg_color_rgb[2]/255.0], 
                     dtype=np.float32)
        
        # Add subtle lighting variation to background (makes it look more natural)
        h, w = bg.shape[:2]
        y, x = np.ogrid[:h, :w]
        center_y, center_x = h//2, w//2
        
        # Create subtle radial gradient for more natural lighting
        max_dist = np.sqrt((h/2)**2 + (w/2)**2)
        distances = np.sqrt((y - center_y)**2 + (x - center_x)**2)
        gradient = 1 - (distances / max_dist) * 0.1  # Very subtle 10% variation
        
        for c in range(3):
            bg[:, :, c] *= gradient
        
        # Step 4: Advanced alpha blending with multiple passes
        alpha_3d = np.expand_dims(alpha_2d, axis=2)
        alpha_3d = np.repeat(alpha_3d, 3, axis=2)
        
        # First pass: Basic compositing
        composite = alpha_3d * spill_corrected_float + (1 - alpha_3d) * bg
        
        # Step 5: Edge refinement with local averaging
        # Apply edge smoothing only in transition areas
        transition_mask = (alpha_2d > 0.01) & (alpha_2d < 0.99)
        if np.any(transition_mask):
            # Create a slightly blurred version for edge smoothing
            try:
                composite_uint8 = (composite * 255).astype(np.uint8)
                blurred_composite = cv2.bilateralFilter(composite_uint8, 7, 50, 50).astype(np.float32) / 255.0
                
                # Blend original and blurred only in transition areas
                blend_factor = np.expand_dims((1 - alpha_2d) * 0.5, axis=2)  # Stronger blending for more transparent areas
                blend_factor = np.repeat(blend_factor, 3, axis=2)
                composite[transition_mask] = composite[transition_mask] * (1 - blend_factor[transition_mask]) + \
                                            blurred_composite[transition_mask] * blend_factor[transition_mask]
            except Exception as e:
                print(f"Edge refinement failed: {e}")
        
        # Step 6: Final quality enhancement
        final_uint8 = (composite * 255).astype(np.uint8)
        
        # Apply very gentle noise reduction to remove any remaining artifacts
        try:
            # Use a small kernel to preserve details while smoothing artifacts
            final_result = cv2.bilateralFilter(final_uint8, 3, 10, 10)
        except Exception as e:
            print(f"Final filtering failed: {e}")
            final_result = final_uint8
        
        return final_result

    def auto_crop_all(self):
        if not self.selected_images:
            messagebox.showwarning("Warning", "Please select images first!")
            return
            
        output_dir = self.output_folder.get()
        os.makedirs(output_dir, exist_ok=True)
        
        thread = threading.Thread(target=self.process_images_thread)
        thread.daemon = True
        thread.start()

    def process_images_thread(self):
        total_images = len(self.selected_images)
        self.progress['maximum'] = total_images
        
        try:
            target_width = int(self.width_var.get())
            target_height = int(self.height_var.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid width and height values!")
            return
            
        success_count = 0
        
        for i, image_path in enumerate(self.selected_images):
            try:
                self.root.after(0, lambda: self.status_label.config(text=f"Processing {i+1}/{total_images}"))
                
                img = Image.open(image_path)
                # Convert to RGB for consistency, then to RGBA only if needed
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                
                cropped_img = self.auto_crop_image(img, self.crop_mode.get(), target_width, target_height)
                cropped_img = cropped_img.resize((target_width, target_height), Image.LANCZOS)
                
                if self.apply_bg_color.get():
                    # Convert to RGBA for background replacement
                    cropped_rgba = cropped_img.convert("RGBA") if cropped_img.mode != 'RGBA' else cropped_img
                    final = self.apply_background_replacement(cropped_rgba)
                else:
                    final = np.array(cropped_img)
                
                save_path = os.path.join(
                    self.output_folder.get(),
                    os.path.splitext(os.path.basename(image_path))[0] + "_auto_crop.png"
                )
                Image.fromarray(final).save(save_path)
                success_count += 1
                
                self.root.after(0, lambda: self.progress.config(value=i+1))
                
            except Exception as e:
                print(f"Error processing {image_path}: {str(e)}")
                continue
                
        self.root.after(0, lambda: messagebox.showinfo("Complete", f"Processed {success_count}/{total_images} images successfully!"))
        self.root.after(0, lambda: self.status_label.config(text="Ready"))
        self.root.after(0, lambda: self.progress.config(value=0))

    def auto_crop_image(self, image, mode, target_width, target_height):
        image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        height, width = image_cv.shape[:2]
        
        if mode == "face":
            gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
            
            # Log each face detected
            for _ in faces:
                print("1 face detected")
            
            if len(faces) > 0:
                largest_face = max(faces, key=lambda x: x[2] * x[3])
                x, y, w, h = largest_face
                
                padding = max(w, h) // 2
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(width, x + w + padding)
                y2 = min(height, y + h + padding)
                
                cropped = image.crop((x1, y1, x2, y2))
                return cropped
                
        elif mode == "body":
            gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
            bodies = self.body_cascade.detectMultiScale(gray, 1.1, 4)
            
            if len(bodies) > 0:
                largest_body = max(bodies, key=lambda x: x[2] * x[3])
                x, y, w, h = largest_body
                
                padding = max(w, h) // 4
                x1 = max(0, x - padding)
                y1 = max(0, y - padding)
                x2 = min(width, x + w + padding)
                y2 = min(height, y + h + padding)
                
                cropped = image.crop((x1, y1, x2, y2))
                return cropped
                
        # Default to center crop
        aspect_ratio = target_width / target_height
        
        if width / height > aspect_ratio:
            new_width = int(height * aspect_ratio)
            x_offset = (width - new_width) // 2
            cropped = image.crop((x_offset, 0, x_offset + new_width, height))
        else:
            new_height = int(width / aspect_ratio)
            y_offset = (height - new_height) // 2
            cropped = image.crop((0, y_offset, width, y_offset + new_height))
            
        return cropped

    def save_cropped_image_manual(self, cropped_image, index):
        try:
            output_dir = self.output_folder.get()
            os.makedirs(output_dir, exist_ok=True)
            
            original_path = self.selected_images[index]
            filename = os.path.basename(original_path)
            name, ext = os.path.splitext(filename)
            
            output_path = os.path.join(output_dir, f"{name}_manual_crop.png")
            success = cv2.imwrite(output_path, cropped_image)
            
            if not success:
                raise Exception(f"Failed to save image to {output_path}")
                
        except Exception as e:
            print(f"Error saving manually cropped image: {str(e)}")
            raise e

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = ImageCropper(root)
        root.mainloop()
    except Exception as e:
        print(f"Error starting application: {str(e)}")
        import traceback
        traceback.print_exc()

1 face detected


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 234GB/s]


1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face detected
1 face d